<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-atomistic-model" data-toc-modified-id="Generate-atomistic-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate atomistic model</a></span></li></ul></div>

## Generate atomistic model

In [7]:
import open3SPN2

# Initialize the DNA from a sequence.
# DNA type can be changed to 'A' or 'B'

seq='ATACAAAGGTGCGAGGTTTCTATGCTCCCACG'
dna=open3SPN2.DNA.fromSequence(seq, dna_type='B_curved',output_pdb=f'initial.pdb')

# Compute the topology for the DNA structure.
# Since the dna was generated from the sequence using X3DNA,
# it is not necesary to recompute the geometry.

dna.computeTopology(template_from_X3DNA=False)

# Create the system.
# To set periodic boundary conditions (periodicBox=[50,50,50]).
# The periodic box size is in nanometers.

dna.periodic=False
s=open3SPN2.System(dna, periodicBox=None)

#Add 3SPN2 forces

s.add3SPN2forces(verbose=True)


Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics


In [8]:
import sys
import simtk.openmm
import simtk.openmm.app
import simtk.unit
import numpy as np

#Initialize Molecular Dynamics simulations

simulation_platform='OpenCL' #['Reference', 'CPU', 'OpenCL', 'CUDA']
temperature=300 * simtk.unit.kelvin
s.initializeMD(temperature=temperature,platform_name=simulation_platform)
simulation=s.simulation

#Set initial positions

simulation.context.setPositions(s.coord.getPositions())

In [9]:
#Obtain total energy

energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

#Obtain detailed energy

energies = {}
for force_name, force in s.forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in s.forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

TotalEnergy -2250.551514 kJ/mol
Bond 2.1e-05 kJ/mol
Angle 0.002319 kJ/mol
Stacking -793.345703 kJ/mol
Dihedral -839.999817 kJ/mol
BasePair -522.301514 kJ/mol
CrossStacking -138.427765 kJ/mol
Exclusion 0.219572 kJ/mol
Electrostatics 43.301292 kJ/mol


In [10]:
#Add simulation reporters

dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)

In [15]:
system_xml=simtk.openmm.XmlSerializer.serialize(s)
with open('temp.xml','w+') as f:
    f.write(system_xml)
    


In [ ]:
simtk.openmm.System.

In [5]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(10000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,-1679.2578125,322.2481053243356
2000,3.999999999999781,-1682.40673828125,306.9846769155262
3000,5.999999999999561,-1700.3797607421875,305.51271863081064
4000,7.999999999999341,-1677.521240234375,309.2116127323974
5000,10.000000000000009,-1655.54248046875,313.8007790242212
6000,12.000000000000677,-1669.7579345703125,284.2262067969661
7000,14.000000000001345,-1639.3497314453125,290.03726981450706
8000,16.00000000000201,-1720.05810546875,286.10434070687154
9000,18.000000000000902,-1680.2728271484375,276.95129035343757
10000,19.999999999999794,-1691.1907958984375,297.7951922786541
